第05课：面向非结构化数据转换的词袋和词向量模型

词袋模型（Bag of Words Model）

In [2]:
import jieba
#定义停用词、标点符号
punctuation = ["，","。", "：", "；", "？"]
#定义语料
content = ["机器学习带动人工智能飞速的发展。",
           "深度学习带动人工智能飞速的发展。",
           "机器学习和深度学习带动人工智能飞速的发展。"
          ]

In [3]:
#分词
segs_1 = [jieba.lcut(con) for con in content]
print(segs_1)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/yw/k7z_d_3567g16ss9plk47x9w0000gn/T/jieba.cache
Loading model cost 0.640 seconds.
Prefix dict has been built succesfully.


[['机器', '学习', '带动', '人工智能', '飞速', '的', '发展', '。'], ['深度', '学习', '带动', '人工智能', '飞速', '的', '发展', '。'], ['机器', '学习', '和', '深度', '学习', '带动', '人工智能', '飞速', '的', '发展', '。']]


In [4]:
tokenized = []
for sentence in segs_1:
    words = []
    for word in sentence:
        if word not in punctuation:          
            words.append(word)
    tokenized.append(words)
print(tokenized)


[['机器', '学习', '带动', '人工智能', '飞速', '的', '发展'], ['深度', '学习', '带动', '人工智能', '飞速', '的', '发展'], ['机器', '学习', '和', '深度', '学习', '带动', '人工智能', '飞速', '的', '发展']]


In [5]:
punctuation

['，', '。', '：', '；', '？']

In [6]:
#求并集
bag_of_words = [ x for item in segs_1 for x in item if x not in punctuation]
#去重
bag_of_words = list(set(bag_of_words))
print(bag_of_words)

['带动', '机器', '发展', '的', '学习', '深度', '和', '人工智能', '飞速']


In [7]:
bag_of_word2vec = []
for sentence in tokenized:
    tokens = [1 if token in sentence else 0 for token in bag_of_words ]
    bag_of_word2vec.append(tokens)


In [8]:
bag_of_word2vec

[[1, 1, 1, 1, 1, 0, 0, 1, 1],
 [1, 0, 1, 1, 1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [9]:
from gensim import corpora
import gensim
#tokenized是去标点之后的
dictionary = corpora.Dictionary(tokenized)
#保存词典
dictionary.save('deerwester.dict') 
print(dictionary)

Dictionary(9 unique tokens: ['人工智能', '发展', '学习', '带动', '机器']...)


In [10]:
#查看词典和下标 id 的映射
print(dictionary.token2id)

{'人工智能': 0, '发展': 1, '学习': 2, '带动': 3, '机器': 4, '的': 5, '飞速': 6, '深度': 7, '和': 8}


In [11]:
# 词向量 （Word Embedding）

使用 One-Hot Encoder有以下问题：

第一，词语编码是随机的，向量之间相互独立，看不出词语之间可能存在的关联关系。
第二，向量维度的大小取决于语料库中词语的多少，如果语料包含的所有词语对应的向量合为一个矩阵的话，那这个矩阵过于稀疏，并且会造成维度灾难。

而解决这个问题的手段，就是使用向量表示（Vector Representations）。比如 Word2Vec 可以将 One-Hot Encoder 转化为低维度的连续值，也就是稠密向量，并且其中意思相近的词也将被映射到向量空间中相近的位置。经过降维，在二维空间中，相似的单词在空间中的距离也很接近。

Word2Vec 主要包含两种模型：Skip-Gram 和 CBOW，值得一提的是，Word2Vec 词向量可以较好地表达不同词之间的相似和类比关系。

In [12]:
from gensim.models import Word2Vec  
import jieba
#定义停用词、标点符号
punctuation = [",","。", ":", ";", ".", "'", '"', "’", "?", "/", "-", "+", "&", "(", ")"]
sentences = [
"长江是中国第一大河，干流全长6397公里（以沱沱河为源），一般称6300公里。流域总面积一百八十余万平方公里，年平均入海水量约九千六百余亿立方米。以干流长度和入海水量论，长江均居世界第三位。",
"黄河，中国古代也称河，发源于中华人民共和国青海省巴颜喀拉山脉，流经青海、四川、甘肃、宁夏、内蒙古、陕西、山西、河南、山东9个省区，最后于山东省东营垦利县注入渤海。干流河道全长5464千米，仅次于长江，为中国第二长河。黄河还是世界第五长河。",
"黄河,是中华民族的母亲河。作为中华文明的发祥地,维系炎黄子孙的血脉.是中华民族民族精神与民族情感的象征。",
"黄河被称为中华文明的母亲河。公元前2000多年华夏族在黄河领域的中原地区形成、繁衍。",
"在兰州的“黄河第一桥”内蒙古托克托县河口镇以上的黄河河段为黄河上游。",
"黄河上游根据河道特性的不同，又可分为河源段、峡谷段和冲积平原三部分。 ",
"黄河,是中华民族的母亲河。"
]

In [13]:
sentences = [jieba.lcut(sen) for sen in sentences]
tokenized = []
for sentence in sentences:
    words = []
    for word in sentence:
        if word not in punctuation:          
            words.append(word)
    tokenized.append(words)

In [14]:
 model = Word2Vec(tokenized, sg=1, size=100,  window=5,  min_count=2,  negative=1, sample=0.001, hs=1, workers=4)

sg=1 是 skip-gram 算法，对低频词敏感；默认 sg=0 为 CBOW 算法。

size 是输出词向量的维数，值太小会导致词映射因为冲突而影响结果，值太大则会耗内存并使算法计算变慢，一般值取为100到200之间。

window 是句子中当前词与目标词之间的最大距离，3表示在目标词前看3-b 个词，后面看 b 个词（b 在0-3之间随机）。

min_count 是对词进行过滤，频率小于 min-count 的单词则会被忽视，默认值为5。

negative 和 sample 可根据训练结果进行微调，sample 表示更高频率的词被随机下采样到所设置的阈值，默认值为 1e-3。

hs=1 表示层级 softmax 将会被使用，默认 hs=0 且 negative 不为0，则负采样将会被选择使用。

In [15]:
model.save('model')  #保存模型
model = Word2Vec.load('model')   #加载模型

In [16]:
print(model.similarity('黄河', '黄河'))

0.99999994


/Users/zhangjianfeng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


In [17]:
print(model.similarity('黄河', '长江'))

0.031182129


/Users/zhangjianfeng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


In [18]:
# 预测最接近的词，预测与黄河和母亲河最接近，而与长江不接近的词
print(model.most_similar(positive=['黄河', '母亲河'], negative=['长江']))

[('中华民族', 0.16356922686100006), ('、', 0.15501049160957336), ('公里', 0.14551666378974915), ('是', 0.12099534273147583), ('于', 0.11948922276496887), ('河道', 0.11142636835575104), ('，', 0.061772894114255905), ('水量', 0.05794735625386238), ('长河', 0.05365785211324692), ('第一', 0.047971852123737335)]


/Users/zhangjianfeng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


Word2Vec 是一种将词变成词向量的工具。通俗点说，只有这样文本预料才转化为计算机能够计算的矩阵向量。



Doc2Vec 是 Mikolov 在 Word2Vec 基础上提出的另一个用于计算长文本向量的工具。在 Gensim 库中，Doc2Vec 与 Word2Vec 都极为相似。但两者在对输入数据的预处理上稍有不同，Doc2vec 接收一个由 LabeledSentence 对象组成的迭代器作为其构造函数的输入参数。其中，LabeledSentence 是 Gensim 内建的一个类，它接收两个 List 作为其初始化的参数：word list 和 label list。

Doc2Vec 也包括两种实现方式：DBOW（Distributed Bag of Words）和 DM （Distributed Memory）。DBOW 和 DM 的实现，二者在 gensim 库中的实现用的是同一个方法，该方法中参数 dm = 0 或者 dm=1 决定调用 DBOW 还是 DM。Doc2Vec 将文档语料通过一个固定长度的向量表达。

In [27]:
#定义数据预处理类，作用是给每个文章添加对应的标签
from gensim.models.doc2vec import Doc2Vec,LabeledSentence
doc_labels = ["长江","黄河","黄河","黄河","黄河","黄河","黄河"]
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
       self.labels_list = labels_list
       self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield LabeledSentence(words=doc,tags=[self.labels_list[idx]])

   

In [28]:
iter_data = LabeledLineSentence(tokenized, doc_labels)

In [29]:
model = Doc2Vec(dm=1, size=100, window=8, min_count=5, workers=4)
model.build_vocab(iter_data)

/Users/zhangjianfeng/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/Users/zhangjianfeng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  # Remove the CWD from sys.path while we load stuff.


In [30]:
tokenized

[['长江',
  '是',
  '中国',
  '第一',
  '大河',
  '，',
  '干流',
  '全长',
  '6397',
  '公里',
  '（',
  '以',
  '沱沱河',
  '为源',
  '）',
  '，',
  '一般',
  '称',
  '6300',
  '公里',
  '流域',
  '总面积',
  '一百八十',
  '余万平方公里',
  '，',
  '年',
  '平均',
  '入海',
  '水量',
  '约',
  '九千',
  '六百余',
  '亿立方米',
  '以',
  '干流',
  '长度',
  '和',
  '入海',
  '水量',
  '论',
  '，',
  '长江',
  '均',
  '居',
  '世界',
  '第三位'],
 ['黄河',
  '，',
  '中国',
  '古代',
  '也',
  '称河',
  '，',
  '发源',
  '于',
  '中华人民共和国',
  '青海省',
  '巴颜喀拉山',
  '脉',
  '，',
  '流经',
  '青海',
  '、',
  '四川',
  '、',
  '甘肃',
  '、',
  '宁夏',
  '、',
  '内蒙古',
  '、',
  '陕西',
  '、',
  '山西',
  '、',
  '河南',
  '、',
  '山东',
  '9',
  '个',
  '省区',
  '，',
  '最后',
  '于',
  '山东省',
  '东营',
  '垦利县',
  '注入',
  '渤海',
  '干流',
  '河道',
  '全长',
  '5464',
  '千米',
  '，',
  '仅次于',
  '长江',
  '，',
  '为',
  '中国',
  '第二',
  '长河',
  '黄河',
  '还是',
  '世界',
  '第五',
  '长河'],
 ['黄河',
  '是',
  '中华民族',
  '的',
  '母亲河',
  '作为',
  '中华文明',
  '的',
  '发祥地',
  '维系',
  '炎黄子孙',
  '的',
  '血脉',
  '是',
  '中华民族',
  '民族',
  '精神',
  '与',


In [31]:
model.train(iter_data,total_examples=model.corpus_count,epochs=1000,start_alpha=0.01,end_alpha =0.001)

/Users/zhangjianfeng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  # Remove the CWD from sys.path while we load stuff.


In [32]:
#根据标签找最相似的，这里只有黄河和长江，所以结果为长江，并计算出了相似度
print(model.docvecs.most_similar('黄河'))


[('长江', 0.9885953664779663)]


In [33]:
print(model.docvecs.similarity('黄河','长江'))

0.9885955


In [34]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')
svm.fit(vec.transform(x_train), y_train)
print(svm.score(vec.transform(x_test), y_test))

NameError: name 'vec' is not defined